In [1]:
import random
from dataclasses import dataclass, asdict
import json

@dataclass
class Move:
    x : int
    y : int
    direction : str

def random_move(dim):
    return Move(
        random.randint(0,dim-2),
        random.randint(0,dim-2),
        random.choice(["ccw","cw"])
    )

def random_moves(dim, n_moves):
    return [random_move(dim) for i in range(n_moves)]

def init_state(dim):
    return [[i*dim+j for j in range(dim)] for i in range(dim)]

def apply_move(b, move):
    dim = len(b)
    x = move.x
    y = move.y

    assert x < dim-1
    assert y < dim-1



    if move.direction == "cw":
        b[x][y], b[x+1][y], b[x+1][y+1], b[x][y+1] = b[x+1][y], b[x+1][y+1], b[x][y+1], b[x][y]
    else:
        b[x][y], b[x+1][y], b[x+1][y+1], b[x][y+1] = b[x][y+1], b[x][y], b[x+1][y], b[x+1][y+1]
        
    return b

def apply_moves(board, moves):
    for move in moves:
        board = apply_move(board, move)
    return board

def print_board(board):
    print("\n".join(",".join(f"{x:>2}" for x in row) for row in board))

def print_move_sequence(dim, moves):
    board = init_state(dim)
    apply_moves(board, moves)
    print_board(board)

def inverse_moves(moves):
    return [Move(m.x,m.y,"ccw" if m.direction == "cw" else "cw") for m in moves[::-1]]

def moves_to_dicts(moves):
    return [asdict(m) for m in moves]


In [227]:
from dataclasses import dataclass
from typing import List
import copy

@dataclass
class Move:
    x: int
    y: int
    direction: str



def solve(board: List[List[int]]) -> List[Move]:
    """Generates a sequence of moves to transform the board to the initial state."""
    board = copy.deepcopy(board)
    dim = len(board)

    def find_position(target: int) -> tuple:
        nonlocal board
        """Find the position (i, j) of the target value in the board."""

        for i in range(len(board)):
            for j in range(len(board[i])):
                if board[i][j] == target:
                    return (i, j)
        raise ValueError(f"Target {target} not found in board")

    moves = []
    def make_move(move):
        nonlocal board, moves
        moves.append(move)
        board = apply_move(board, move)
        return board

    def move_left(fr):
        i,j = fr
        if i == dim-1:
            make_move(Move(i-1,j-1, "cw"))
        else:
            make_move(Move(i,j-1, "ccw"))
    
    def move_up(fr):
        i,j = fr
        if j == dim-1:
            make_move(Move(i-1,j-1, "ccw"))
        else:
            make_move(Move(i-1,j, "cw"))

    def move_right(fr):
        i,j = fr
        if i == dim-1:
            make_move(Move(i-1,j, "ccw"))
        else:
            make_move(Move(i,j, "cw"))
    
    def move_down(fr):
        i,j = fr
        if j == dim-1:
            make_move(Move(i,j-1, "cw"))
        else:
            make_move(Move(i,j, "ccw"))

    def move_to(fr,to):
        while fr[0] < to[0]:
            move_down(fr)
            fr = (fr[0]+1, fr[1])
        while fr[1] < to[1]:
            move_right(fr)
            fr = (fr[0], fr[1]+1)
        while fr[0] > to[0]:
            move_up(fr)
            fr = (fr[0]-1, fr[1])
        while fr[1] > to[1]:
            move_left(fr)
            fr = (fr[0], fr[1]-1)
        
    for layer in range(dim-2):
        for col in range(layer,dim-2):
            target = layer * dim + col
            move_to(find_position(target),(layer,col))
        
        move_to(find_position(layer*dim+dim-1),(layer,dim-2))

        if find_position(layer*dim+dim-2) == (layer,dim-1):
            make_move(Move(layer,dim-2, "cw"))
            make_move(Move(layer+1,dim-2, "ccw"))
            make_move(Move(layer,dim-2, "ccw"))
        move_to(find_position(layer*dim+dim-2),(layer+1,dim-2))
        
        make_move(Move(layer,dim-2, "cw"))


        for row in range(layer,dim-2):
            target = row * dim + layer
            move_to(find_position(target),(row,layer))
        
        move_to(find_position((dim-1)*dim+layer),(dim-2,layer))

        if find_position((dim-2)*dim+layer) == (dim-1,layer):
            make_move(Move(dim-2,layer, "ccw"))
            make_move(Move(dim-2,layer+1, "cw"))
            make_move(Move(dim-2,layer, "cw"))
        move_to(find_position((dim-2)*dim+layer),(dim-2,layer+1))
        make_move(Move(dim-2,layer, "ccw"))
    return moves

In [228]:
import copy
import random
correct = 0
for seed in range(0,200):
    random.seed(seed)
    dim = 10
    board = init_state(dim)

    init_board = apply_moves(board, random_moves(dim, 1000))
    board = copy.deepcopy(init_board)
    print_board(init_board)
    sol_moves = solve(board)
    board = apply_moves(init_board, sol_moves)
    print_board(board)

    if board == init_state(dim):
        correct += 1

print(correct)

60,27, 0, 6, 2,29,16,21,66,17
 4,43,91,24,52,48, 3,15,57,58
32,30,53,13,11,20,50,68,31,72
 9,28,44,77,45, 5,34,18,49,54
25,95, 8,33,96,55,22, 7,64,86
51,79,12,35,88,37,62,71,97,56
74,83,23,73,85,47,70,65,93,63
75,90,61, 1,84,82,98,38,87,10
19,92,42,99,36,14,89,39,41,59
69,94,67,40,26,78,76,80,81,46
 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
10,11,12,13,14,15,16,17,18,19
20,21,22,23,24,25,26,27,28,29
30,31,32,33,34,35,36,37,38,39
40,41,42,43,44,45,46,47,48,49
50,51,52,53,54,55,56,57,58,59
60,61,62,63,64,65,66,67,68,69
70,71,72,73,74,75,76,77,78,79
80,81,82,83,84,85,86,87,98,89
90,91,92,93,94,95,96,97,88,99
 1,11,94,42, 0,90,26,16,49, 7
47,30, 3,22,21,17,46,15,99,66
55,20,73, 2,32,12,27, 4, 8, 9
85,72,60,82,59,24,57,51,69,38
91,31,52,18,61,10,84,37,79,19
75,83,28,45,74,40,78,14,25,93
54,13,29,98,48,65,41,70,97,88
33,80,64,96,56,39,87,34,76,89
23,95,53,43,36,92,81,44,58, 6
71,35,63,68, 5,67,86,50,62,77
 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
10,11,12,13,14,15,16,17,18,19
20,21,22,23,24,25,26,27,28,29
30,31,32,3

In [229]:
def get_cycles(board):
    dim = len(board)
    seen = [0]*dim*dim
    ret = []
    flat = sum(board,[])
    for i in range(dim*dim):
        if seen[i]:
            continue
        x = i
        cyc = []
        while True:
            x = flat[x]
            if seen[x]:
                break
            cyc.append(x)
            seen[x] = True
        ret.append(cyc)
    return ret

In [230]:
dim = 20
a = random_moves(dim, random.randint(1000,2000))
x = random_moves(dim, random.randint(1000,2000))
board_x = apply_moves(init_state(dim), x)
board_y = apply_moves(init_state(dim), a + x + inverse_moves(a))
print("Here's a random conjugancy problem:")
print("X =")
print_board(board_x)
print()
print("Y =")
print_board(board_y)
print()
print("Give a list of moves S such that S*X*S^{-1} = Y")

Here's a random conjugancy problem:
X =
86,20,105,84, 5,43,44,28,42, 9,30,191,91,53,15,12,39,38,18,59
21,100, 1,40,24,49,32,66,69, 7,47, 8,114,95,75,17,37,55,79,19
61,64,83,45, 6,26, 4,88,89,29,108,31,113,92,51,35,13,58,77,98
23,22,122,65, 2, 0,67, 3,143,11,48,209,109,52,115,56,16,78,57,14
63,165,187,225,85,244,25,149,87,93,127,74,70,135,54,76,216,110,139,99
102,101,161,50,81,103,71,131,151,10,27,72,130,137,132,178,154,157,116,119
162,62,80,41,124,125,163,186,148,68,153,150,34,33,174,239,134,96,97,297
183,140,120,123,205,60,141,152,107,228,173,46,237,234,147,94,138,218,136,156
221,121,222,181,104,210,171,129,166,111,212,90,197,190,193,133,179,176,117,118
180,182,226,207,204,201,146,223,73,308,128,208,196,293,213,112,250,36,158,155
203,160,188,260,285,106,185,247,189,192,274,229,275,249,214,313,256,198,199,219
184,82,200,142,145,126,370,248,232,170,287,352,252,215,194,312,277,296,217,257
242,261,280,264,281,331,246,266,164,364,168,311,169,289,177,253,235,175,358,238
240,227,262,241,220,

In [ ]:
cycles_x = sorted(get_cycles(board_x),key=lambda x: len(x))
cycles_y = sorted(get_cycles(board_y),key=lambda x: len(x))
print(cycles_x)
print(cycles_y)

for offset in range(100):
    permutation = [0]*dim*dim
    for cyc_x, cyc_y in zip(cycles_x, cycles_y):
        for i in range(len(cyc_y)):
            permutation[cyc_x[i]] = cyc_y[(i+offset)%len(cyc_y)]

    board_S = [permutation[i:i+dim] for i in range(0,dim*dim,dim)]
    
    s_moves = solve(board_S)
    s_moves = inverse_moves(s_moves)
    
    if board_S == apply_moves(init_state(dim), s_moves):
        break

apply_moves(init_state(dim),s_moves + x + inverse_moves(s_moves)) == board_y

[[9], [18], [24], [99], [101], [119], [124], [125], [180], [214], [219], [246], [262], [279], [353], [362], [44, 6], [365, 363, 361], [58, 77, 78, 57], [390, 391, 392, 351], [131, 150, 173, 190, 128, 148, 107], [30, 47, 88, 87, 149, 228, 232, 252, 169, 111, 72, 109, 10], [86, 25, 49, 29, 7, 28, 69, 11, 191, 208, 189, 308, 292, 290, 369, 381, 301, 320, 343, 383, 388, 230, 287, 307, 224, 145, 60, 23, 40, 61, 22, 1, 20, 21, 100, 102, 161, 121, 62, 122, 80, 63, 65, 0], [42, 83, 225, 126, 163, 181, 182, 226, 370, 348, 269, 326, 329, 333, 251, 311, 371, 332, 335, 334, 291, 288, 347, 286, 330, 272, 172, 197, 36, 37, 55, 35, 17, 38, 79, 14, 15, 12, 91, 74, 115, 178, 117, 157, 218, 199, 155, 94, 54, 51, 31, 8], [105, 103, 50, 108, 151, 46, 4, 5, 43, 45, 26, 32, 114, 132, 34, 75, 56, 13, 53, 92, 70, 48, 89, 93, 135, 239, 257, 175, 133, 33, 95, 76, 16, 39, 19, 59, 98, 139, 297, 278, 258, 358, 337, 298, 259, 238, 217, 198, 158, 136, 134, 174, 193, 293, 211, 229, 170, 212, 275, 316, 379, 357, 355, 

True